In [2]:
import numpy as np
import scipy.optimize as opt
from scipy.optimize import shgo
from sympy import symbols, diff, solve, hessian

In [3]:
def f(x,c):
    return x**4 - x**2 - c*x

def f1(x):
    return f(x,0.0005)

In [4]:
def f0(x):
    return x**4 - x**2


In [5]:
x0 = -1
sol = opt.minimize(f0, x0)
print("Minimum at:", sol.x)

Minimum at: [-0.70710652]


In [6]:
x0 = -1
sol2 = opt.minimize(f1, x0)
print("Minimum at:", sol2.x)

Minimum at: [0.70723222]


In [7]:
sol3 = shgo(f0, [(-5, 5)], sampling_method='sobol')
print("Global minimum at:", sol3.x)

Global minimum at: [0.70710724]


In [8]:
def f0(x):
    x = x[0]
    return x**4 - x**2

bounds = [(-2, 2)]

res = shgo(f0, bounds, sampling_method='sobol')  # sobol is a good sampler
print("res.x (one minimizer):", res.x)        # single global minimizer returned
print("res.fun:", res.fun)                    # function value at res.x
print("res.xl (all minima found):")
print(res.xl)                                 # list of all local minima points found
print("res.funl (values at each local minima):")
print(res.funl)

res.x (one minimizer): [0.70710679]
res.fun: -0.2499999999999999
res.xl (all minima found):
[[ 0.70710679]
 [-0.7071068 ]]
res.funl (values at each local minima):
[-0.25 -0.25]


In [9]:
len(res.xl)

2

In [10]:
for c in [0.0, 0.0001, 0.0005, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 1, 1.5, 2]:
    resultado = shgo(lambda x: f(x[0], c), [(-2, 2)], sampling_method='sobol')
    if len(resultado.xl) > 1:
        print(f"c = {c}: Multiple minima found at {resultado.xl} with values {resultado.funl}")
    elif len(resultado.xl) == 1:
        print(f"c = {c}: Single minimum found at {resultado.xl[0]} with value {resultado.funl[0]}")

c = 0.0: Multiple minima found at [[ 0.70710679]
 [-0.7071068 ]] with values [-0.25 -0.25]
c = 0.0001: Multiple minima found at [[ 0.70713178]
 [-0.7070818 ]] with values [-0.25007071 -0.24992929]
c = 0.0005: Multiple minima found at [[ 0.70723175]
 [-0.70698177]] with values [-0.25035358 -0.24964648]
c = 0.001: Multiple minima found at [[ 0.70735665]
 [-0.70685667]] with values [-0.25070723 -0.24929302]
c = 0.01: Multiple minima found at [[ 0.70959364]
 [-0.70459344]] with values [-0.25708352 -0.24294148]
c = 0.1: Multiple minima found at [[ 0.73089308]
 [-0.68063928]] with values [-0.32191935 -0.18058697]
c = 0.2: Multiple minima found at [[ 0.7526187]
 [-0.650488 ]] with values [-0.39611014 -0.11399412]
c = 0.3: Multiple minima found at [[ 0.77269818]
 [-0.61482917]] with values [-0.47238833 -0.05067089]
c = 0.4: Multiple minima found at [[ 0.79142546]
 [-0.56959284]] with values [-0.55060479  0.00865985]
c = 0.5: Multiple minima found at [[ 0.80901665]
 [-0.5       ]] with values [

In [11]:
##Let us do a proper example: V(x,y) = 1/4(x^2 +y^2 - 1)^2 + c/2 x^2

x, y = symbols('x y')
c = symbols('c')
V = 1/4*(x**2 + y**2 - 1)**2 + c/2*x**2


V_x = diff(V, x)
V_y = diff(V, y)

critical_points = solve([V_x, V_y], (x, y))

In [12]:
critical_points

[(0.0, -1.00000000000000),
 (0.0, 0.0),
 (0.0, 1.00000000000000),
 (-sqrt(1.0 - c), 0.0),
 (sqrt(1.0 - c), 0.0)]

In [13]:
hes = hessian(V, (x, y))
hes

Matrix([
[c + 3.0*x**2 + 1.0*y**2 - 1.0,                   2.0*x*y],
[                      2.0*x*y, 1.0*x**2 + 3.0*y**2 - 1.0]])

In [14]:
det_hes = hes.det()
det_hes.subs({x: critical_points[0][0], y: critical_points[0][1]})

2.0*c

In [15]:
stab = []
for i in range(len(critical_points)):
    hes_cp = det_hes.subs({x: critical_points[i][0], y: critical_points[i][1]})
    stab.append(hes_cp)
    print(f"Critical Point {i+1}: {critical_points[i]}, Determinant of Hessian: {hes_cp}")
stab

Critical Point 1: (0.0, -1.00000000000000), Determinant of Hessian: 2.0*c
Critical Point 2: (0.0, 0.0), Determinant of Hessian: 1.0 - 1.0*c
Critical Point 3: (0.0, 1.00000000000000), Determinant of Hessian: 2.0*c
Critical Point 4: (-sqrt(1.0 - c), 0.0), Determinant of Hessian: 1.0*c*(1.0 - c) + 3.0*c + 3.0*(1.0 - c)**2 - 3.0
Critical Point 5: (sqrt(1.0 - c), 0.0), Determinant of Hessian: 1.0*c*(1.0 - c) + 3.0*c + 3.0*(1.0 - c)**2 - 3.0


[2.0*c,
 1.0 - 1.0*c,
 2.0*c,
 1.0*c*(1.0 - c) + 3.0*c + 3.0*(1.0 - c)**2 - 3.0,
 1.0*c*(1.0 - c) + 3.0*c + 3.0*(1.0 - c)**2 - 3.0]

In [32]:
c_values = np.linspace(-1, 1, 1000)

prev_cp = None
prev_c = None
prev_h = None

for i in range(len(stab)):
    prev_cp = critical_points[i]
    prev_h = stab[i].subs(c, c_values[0])
    prev_c = c_values[0]

    for c_val in c_values[1:]:
        h = stab[i].subs(c, c_val)

        if (prev_h > 0 > h) or (prev_h < 0 < h):
            print(f"Critical Point: {prev_cp}, c value: {prev_c}, Determinant of Hessian: {prev_h}")

        prev_h = h
        prev_c = c_val


Critical Point: (0.0, -1.00000000000000), c value: -0.0010010010010009784, Determinant of Hessian: -0.00200200200200196
Critical Point: (0.0, 1.00000000000000), c value: -0.0010010010010009784, Determinant of Hessian: -0.00200200200200196
Critical Point: (-sqrt(1.0 - c), 0.0), c value: -0.0010010010010009784, Determinant of Hessian: 0.00200400600801021
Critical Point: (sqrt(1.0 - c), 0.0), c value: -0.0010010010010009784, Determinant of Hessian: 0.00200400600801021


In [ ]:
'''Now, I shall move to the actual case!'''

999

In [33]:
'''
Definitions:

x = S_R
y = S_I
z = H

'''

'\nDefinitions:\n\nx = S_R\ny = S_I\nz = H\n\n'

In [38]:
clear(x, y, V)

In [57]:
x, y, z = symbols('x y z')

#Potential Symbols:

m, lambdaS, lambda_H, lambda_HS, lambda1, param = symbols('m lambda_S lambda_H lambda_P lambda_1 P', real=True, positive=True)


v_scalar = -m**2/2 * (x**2 - y**2) + lambdaS/4 * (x**4 + y**4 + 2*(x**2)*(y**2))+ lambda_H * z**4 + lambda_HS/4 * (x**2 + y**2) * z**2 + lambda1/2 * (x**4 + y**4 - 6*(x**2)*(y**2)) + 2*param*(x*y)*(y**2 - x**2)

In [58]:
v_scalar

2*P*x*y*(-x**2 + y**2) + lambda_1*(x**4 - 6*x**2*y**2 + y**4)/2 + lambda_H*z**4 + lambda_P*z**2*(x**2 + y**2)/4 + lambda_S*(x**4 + 2*x**2*y**2 + y**4)/4 - m**2*(x**2 - y**2)/2

In [ ]:
#Let us see if it finds the minima correctly

v_scalar_x = diff(v_scalar, x)
v_scalar_y = diff(v_scalar, y)
v_scalar_z = diff(v_scalar, z)

critical_points = solve([v_scalar_x, v_scalar_y, v_scalar_z], (x, y, z))